In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat  = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)
template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")


prompt = template.format(country_a="maxico", country_b="korea")
chat.predict(prompt)


In [ ]:
#SystemMessage는 LLM에 설정들을 제공하기 위한 메시지임.

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And tou only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human","What is the distance between {country_a} and {country_b}. Also, What is your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="France",
    country_b="Korea",
)

chat.predict_messages(prompt)

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",") #앞 뒤 공백 잘라줌, 쉼표기준으로 잘라 array로 저장
        return list(map(str.strip,items))

p = CommaOutputParser()
p.parse("Hello,how ,are, you ")

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everyting you are will be answerd with a comma seperated list of max {max_items} in lowercase. Do NOT replay with anything else."),
    ("human", "{question}")
    ]
)

In [ ]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items" : 5,
    "question" : "What are the pokemons?"
})